# NVIDIA Driver

You'll need a functional nvidia compute capability 5.3 or better card. This basically means
Testa P series or Geforce GTX 10 series or better. See https://developer.nvidia.com/cuda-gpus
for more details.

Ensure nvidia-smi can see your gpu. Should look something like this:
    
```
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 435.21       Driver Version: 435.21       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 106...  Off  | 00000000:01:00.0  On |                  N/A |
|  0%   35C    P8     7W / 120W |    454MiB /  6075MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
```

In [1]:
!nvidia-smi

Sun Jan  5 12:45:02 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.87.01    Driver Version: 418.87.01    CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:1E.0 Off |                    0 |
| N/A   30C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

# TensorFlow GPU support

Ensure tensorflow has functional gpu support. You should have a GPU line 
referencing the GPU identified by nvidia-smi above.

In [2]:
import tensorflow as tf
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")
    
with tf.device(tf.test.gpu_device_name()):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
    c = tf.matmul(a, b)
    # Creates a session with log_device_placement set to True.
    sess = tf.Session(config=tf.ConfigProto())
    # Runs the op.
    r = sess.run(c)
    print(r)
    import numpy as np
    assert np.all(r == np.array([[22., 28.], [49., 64.]]))

Default GPU Device: /device:GPU:0
[[22. 28.]
 [49. 64.]]


# Working Directories 

In [4]:
import os
from os import makedirs
from os.path import isdir

readable = ["../data"]
writable = [
    "../src",
    "../scratch"
]

for f in writable:
    makedirs(f, exist_ok=True)
    
for f in readable + writable:
    print("\tdirectory:", isdir(f), "readable:", os.access(f, os.R_OK), "writable:", os.access(f, os.W_OK), "\t", f)

	directory: True readable: True writable: False 	 ../data
	directory: True readable: True writable: True 	 ../src
	directory: True readable: True writable: True 	 ../scratch
